In [1]:
# -*- coding: utf-8 -*-

# Sample Python code for youtube.playlistItems.list
# See instructions for running these code samples locally:
# https://developers.google.com/explorer-help/guides/code_samples#python

import os

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

from IPython.display import clear_output

from utils import Heartbeat

scopes = [
    "https://www.googleapis.com/auth/youtubepartner",
    "https://www.googleapis.com/auth/youtube",
    "https://www.googleapis.com/auth/youtube.force-ssl",
]
reqFreq = 1.0/2

In [2]:
# Setup
os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"
api_service_name    = "youtube"
api_version         = "v3"
client_secrets_file = "../keys/client_secrets.json"

def get_flow_credentials_yt( client_secrets_file, scopes ):
    # Get credentials and create an API client
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
        client_secrets_file, 
        scopes
    )
    credentials = flow.run_console()
    youtube = googleapiclient.discovery.build(
        api_service_name, 
        api_version, 
        credentials = credentials
    )
    clear_output( wait = False ) # Prevent keys from being stored in output
    return flow, credentials, youtube

flow, credentials, youtube = get_flow_credentials_yt( client_secrets_file, scopes )

In [3]:
from pprint import pprint
from random import shuffle

def fetch_entire_playlist( playlistId, part ):
    """ Send serial requests until info about all playlist items is obtained """
    
    limiter = Heartbeat( reqFreq )
    
    # Init
    request = youtube.playlistItems().list(
        playlistId = playlistId ,
        part       = part ,
        maxResults = 50
    )
    response = request.execute()
    nextPage = response["nextPageToken"]
    items    = response['items']
    last     = False
    
    while 1:
        request = youtube.playlistItems().list(
            playlistId = "PLxgoClQQBFjg29FkEQ_7rfDimHFzfFe9i",
            part       = "contentDetails,id,snippet,status",
            pageToken  = nextPage,
            maxResults = 50
        )
        response = request.execute()
        try:
            nextPage = response["nextPageToken"]
        except KeyError:
            last = True
        items.extend( response['items'] )
        if last:
            break
        else:
            limiter.rest()
        
    return items


def reorder_entire_playlist( itemList ):
    """ Given a list of video dictionaries, reorder them randomly in their parent playlist """
    N    = len( itemList )
    posn = list( range( N-1, -1, -1 ) )
    sccs = 0
    
    shuffle( itemList )
    
    limiter = Heartbeat( reqFreq )
    
    for i, item in enumerate( itemList ):
        # Notify
        print( f'\n=== Item {i+1} of {N} ===\n' )
        # Reorder
        print( f'\tMoving {item["snippet"]["position"]} --to-> {posn[i]} ' )
        request = youtube.playlistItems().update(
            part = 'id,snippet',
            body = {
                'id': item['id'],
                'snippet': {
                    'playlistId': item['snippet']['playlistId'],
                    'resourceId': item['snippet']['resourceId'],
                    'position'  : posn[i],
                }
            }
        )
        response = request.execute()
        # Check
        if response['snippet']['position'] == posn[i]:
            sccs += 1
            print( f'SUCCESS: {response["snippet"]["position"]}' )
        else:
            print( f'FAILURE: {response["snippet"]["position"]}, desired {posn[i]}' )
        limiter.rest()
        
    # Notify
    print( f'Success: {sccs} / {N}' )
    
    

In [4]:
playlistId = "PLxgoClQQBFjg29FkEQ_7rfDimHFzfFe9i" 
part       = "contentDetails,id,snippet,status" 
items = fetch_entire_playlist( playlistId, part )

N      = len( items )
public = 0

for res in items:
    if res['status']['privacyStatus'] == 'public':
        public += 1
    else:
        print( f"Nonpublic Video: {res['status']['privacyStatus']}" )

print( f"{public} / {N} items are public!" )
        

HttpError: <HttpError 403 when requesting https://youtube.googleapis.com/youtube/v3/playlistItems?playlistId=PLxgoClQQBFjg29FkEQ_7rfDimHFzfFe9i&part=contentDetails%2Cid%2Csnippet%2Cstatus&maxResults=50&alt=json returned "The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.". Details: "[{'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'domain': 'youtube.quota', 'reason': 'quotaExceeded'}]">

In [ ]:
print( '\n\n==== Reorder! ====\n' )
reorder_entire_playlist( items )
    